In [1]:
import polars as pl
import os
import duckdb

In [2]:
AUXILIAR_PATH = r'finding-home/finding_home/auxiliar/'

# IMPORTAÇÃO DE DADOS

## CIDADES

In [23]:
from unicodedata import normalize 
def custnorm(In_series):
    for i, x in enumerate(In_series):
        newvalue = normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8')
        if newvalue != x:
            In_series[i]=newvalue
    return In_series

In [24]:
cities = pl.read_csv(os.path.join(AUXILIAR_PATH, 'CIDADES_LITORANEAS_IBGE.csv'), separator=';') \
           .filter(pl.col('NM_REGIAO').is_in(['Sul', 'Sudeste', 'Nordeste'])) \
           .with_columns(pl.col('NM_MUN').str.to_uppercase().alias('MUNICIPIO')) \
           .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))

/tmp/ipykernel_134355/3017128636.py:4: DeprecationWarning: `map` is deprecated. It has been renamed to `map_batches`.
  .with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIO'))


In [25]:
cities.head()

CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,NM_REGIAO,CD_UF,NM_UF,MUNICIPIO
i64,str,str,f64,str,i64,str,str
2100204,"""Alcântara""","""MA""",1167.964,"""Nordeste""",21,"""Maranhão""","""ALCANTARA"""
2100832,"""Apicum-Açu""","""MA""",341.12,"""Nordeste""",21,"""Maranhão""","""APICUM-ACU"""
2100907,"""Araioses""","""MA""",1789.73,"""Nordeste""",21,"""Maranhão""","""ARAIOSES"""
2101103,"""Axixá""","""MA""",160.462,"""Nordeste""",21,"""Maranhão""","""AXIXA"""
2101251,"""Bacabeira""","""MA""",542.962,"""Nordeste""",21,"""Maranhão""","""BACABEIRA"""


## HOSPITAIS

In [7]:
hospitals = pl.read_csv(os.path.join(AUXILIAR_PATH, 'hospitais.csv'), separator='|', encoding='cp1252', truncate_ragged_lines=True)

In [26]:
hospitals.columns

['',
 'COMP',
 'REGIAO',
 'UF',
 'MUNICIPIO',
 'MOTIVO_DESABILITACAO',
 'CNES',
 'NOME_ESTABELECIMENTO',
 'RAZAO_SOCIAL',
 'TP_GESTAO',
 'CO_TIPO_UNIDADE',
 'DS_TIPO_UNIDADE',
 'NATUREZA_JURIDICA',
 'DESC_NATUREZA_JURIDICA',
 'NO_LOGRADOURO',
 'NU_ENDERECO',
 'NO_COMPLEMENTO',
 'NO_BAIRRO',
 'CO_CEP',
 'NU_TELEFONE',
 'NO_EMAIL',
 'LEITOS_EXISTENTES',
 'LEITOS_SUS',
 'UTI_TOTAL_EXIST',
 'UTI_TOTAL_SUS',
 'UTI_ADULTO_EXIST',
 'UTI_ADULTO_SUS',
 'UTI_PEDIATRICO_EXIST',
 'UTI_PEDIATRICO_SUS',
 'UTI_NEONATAL_EXIST',
 'UTI_NEONATAL_SUS',
 'UTI_QUEIMADO_EXIST',
 'UTI_QUEIMADO_SUS',
 'UTI_CORONARIANA_EXIST',
 'UTI_CORONARIANA_SUS']

In [27]:
hospitals_stat = duckdb.sql("""
                            SELECT 
                                    REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HOSPITAL_GERAL
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS PRONTO_SOCORRO_GERAL 
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS UNIDADE_MISTA
                            
                            FROM hospitals
                            GROUP BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA

                            ORDER BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                            
                            """).to_df()

In [28]:
hospitals_stat.head()

,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,PRONTO_SOCORRO_GERAL,UNIDADE_MISTA
0,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_FILANTRÓPICO,4.0,0.0,0.0
1,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PRIVADO,26.0,0.0,0.0
2,CENTRO-OESTE,DF,BRASILIA,HOSPITAL_PÚBLICO,16.0,0.0,0.0
3,CENTRO-OESTE,GO,ABADIANIA,HOSPITAL_PÚBLICO,0.0,0.0,1.0
4,CENTRO-OESTE,GO,ACREUNA,HOSPITAL_PRIVADO,2.0,0.0,0.0


In [22]:
duckdb.sql("SELECT * FROM hospitals_stat WHERE MUNICIPIO LIKE '%APICUM-ACU%'")

┌──────────┬────────────┬────────────────────────┬────────────────┬──────────────────────┬───────────────┐
│  REGIAO  │ MUNICIPIO  │ DESC_NATUREZA_JURIDICA │ HOSPITAL_GERAL │ PRONTO_SOCORRO_GERAL │ UNIDADE_MISTA │
│ varchar  │  varchar   │        varchar         │     double     │        double        │    double     │
├──────────┼────────────┼────────────────────────┼────────────────┼──────────────────────┼───────────────┤
│ NORDESTE │ APICUM-ACU │ HOSPITAL_PÚBLICO       │            1.0 │                  0.0 │           0.0 │
└──────────┴────────────┴────────────────────────┴────────────────┴──────────────────────┴───────────────┘

In [20]:
cities = cities.with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIOS'))

/tmp/ipykernel_134355/948987951.py:1: DeprecationWarning: `map` is deprecated. It has been renamed to `map_batches`.
  cities = cities.with_columns(pl.col('MUNICIPIO').map(custnorm).alias('MUNICIPIOS'))


In [21]:
cities.head()

CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,NM_REGIAO,CD_UF,NM_UF,MUNICIPIO,MUNICIPIOS
i64,str,str,f64,str,i64,str,str,str
2100204,"""Alcântara""","""MA""",1167.964,"""Nordeste""",21,"""Maranhão""","""ALCÂNTARA""","""ALCANTARA"""
2100832,"""Apicum-Açu""","""MA""",341.12,"""Nordeste""",21,"""Maranhão""","""APICUM-AÇU""","""APICUM-ACU"""
2100907,"""Araioses""","""MA""",1789.73,"""Nordeste""",21,"""Maranhão""","""ARAIOSES""","""ARAIOSES"""
2101103,"""Axixá""","""MA""",160.462,"""Nordeste""",21,"""Maranhão""","""AXIXÁ""","""AXIXA"""
2101251,"""Bacabeira""","""MA""",542.962,"""Nordeste""",21,"""Maranhão""","""BACABEIRA""","""BACABEIRA"""
